In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS

from dotenv import load_dotenv
from operator import itemgetter 
import os

from tqdm import tqdm

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [6]:
def load_vdb_and_retriever(path="../vectorstore/historia_ed_financeira",
                           k=4):
    embedding_size = 1536
    embedding_model = "text-embedding-3-small"
    embeddings = OpenAIEmbeddings(model=embedding_model, dimensions=embedding_size)
    
    vdb = FAISS.load_local(path, 
                           embeddings, 
                           allow_dangerous_deserialization=True)
    
    retriever = vdb.as_retriever(search_kwargs={"k": k})
    
    return vdb, retriever
    

### RAG (Retrieval-Augmented Generation)

A técnica utilizada aqui consiste em utilizar o mecanismo de busca baseado nos ``embeddings`` para que se encontre os documentos mais próximos semanticamente para a geração de texto. Dessa forma, o modelo consegue gerar textos que se baseiam na informação contida nos documentos mais próximos.

<img src="../imgs/rag.png" width="600">

In [7]:
vdb, retriever = load_vdb_and_retriever()

retriever.invoke("Qual o peso de um sonho?")

[Document(id='2a0b4941-1bd8-4d17-bbe0-bb5e89b6aa5e', metadata={'source': '../data/educacao_financeira.pdf', 'page': 48}, page_content='3636\nO que pesa mais, um quilo de algodão ou um quilo de chum-\nbo? Muita gente responde chumbo ao ouvir esta pergunta, \nmas, na verdade, os dois pesam a mesma coisa: um quilo. \nNa hora de pensar nas nossas despesas, também podemos \nficar confusos e acabar dando pesos equivocados a cada coi-\nsa. Às vezes, achamos que estamos gastando muito em uma \ncoisa, quando o que está pesando no nosso bolso na verdade \né outra. \nquaNTO pesa O seu sONhO? \nVocê e sua família devem ter alguns sonhos e projetos que pa-\nrecem muito distantes porque nunca sobra dinheiro suficien-\nte no final do mês para realizá-los. O que fazer nesse caso?\nAntes de tudo, seria bom vocês elegerem um dos projetos que \nquerem realizar para ter um foco bem concreto. Isso ajuda a \nmanter o esforço da família para conseguir o dinheiro neces-\nsário. Escolheu um foco?\nAgora você v

In [9]:
from langchain_core.runnables import RunnableLambda
from langchain_core.documents import Document
from typing import List, Dict

def format_docs(docs: List[Document]) -> str:
    return "\n".join([x.page_content for x in docs])

vdb, retriever = load_vdb_and_retriever()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
# llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)


system_prompt = """
Você é um assistente de IA que vai tirar dúvidas sobre educação financeira e história. 

Além disso, aqui está um conteudo extra sobre educação financeira e/ou história:

[Conteudo extra]
{extra_content}
[Final do conteudo extra]

--------------------------------------------
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

chain = (
    {
        "input": itemgetter("input"),
        "extra_content": itemgetter("input") | retriever | RunnableLambda(format_docs)
    }
    | prompt 
    | llm 
    | StrOutputParser()
)

In [10]:
response = chain.invoke({"input": "Me fale sobre o peso de um sonho"})

In [11]:
print(response)

O "peso de um sonho" é uma metáfora que se refere à importância e ao impacto que um sonho ou objetivo pode ter na vida de uma pessoa. Assim como um quilo de algodão e um quilo de chumbo pesam a mesma coisa, os sonhos podem parecer diferentes em termos de dificuldade ou viabilidade, mas todos têm um peso emocional e prático.

Quando falamos sobre o peso de um sonho, podemos considerar alguns aspectos:

1. **Prioridade**: Alguns sonhos são mais importantes do que outros e, portanto, têm um "peso" maior em nossas vidas. É essencial identificar quais sonhos são prioritários e merecem nosso foco e esforço.

2. **Planejamento**: Para realizar um sonho, é necessário um planejamento financeiro e pessoal. Isso envolve entender o que é necessário para alcançá-lo e como podemos nos organizar para isso.

3. **Sacrifícios**: Muitas vezes, para realizar um sonho, precisamos abrir mão de outras coisas. O peso do sonho pode ser medido em termos de sacrifícios que estamos dispostos a fazer.

4. **Impac

In [12]:
async for chunk in chain.astream({"input": "Me fale sobre o peso de um sonho"}):
    print(chunk, end="", flush=True)

O "peso de um sonho" é uma metáfora que se refere à importância e ao impacto que um sonho ou objetivo pode ter na vida de uma pessoa. Assim como na famosa pergunta sobre o peso de um quilo de algodão versus um quilo de chumbo, onde ambos pesam a mesma coisa, os sonhos podem parecer diferentes em termos de dificuldade ou importância, mas todos têm um peso emocional e prático em nossas vidas.

Quando falamos sobre o peso de um sonho, podemos considerar alguns aspectos:

1. **Importância Pessoal**: Cada sonho tem um significado único para quem o possui. Um sonho pode ser um desejo profundo de mudança, realização ou conquista, e seu peso pode ser medido pela motivação que ele gera.

2. **Desafios e Sacrifícios**: Realizar um sonho muitas vezes exige esforço, dedicação e, em muitos casos, sacrifícios. O peso do sonho pode ser sentido nas dificuldades enfrentadas ao longo do caminho.

3. **Planejamento Financeiro**: Muitos sonhos requerem recursos financeiros para serem realizados. O peso do